# Project Obejctive:

### The project aims to apply the principles of Forward and Inverse Kinematics to a Robot Manipulator. CoppeliaSim was used as the simulation tool where the UR5 robot manipulator was used as an example. The robot is capable of preforming both Forward and Inverse Kinematics, as well as simple Linear Interpolation. The script can be easily edited to allow for more complex tasks such as pick and place.

In [2]:
import numpy as np
import time
from Kinematics import *
from Trajectory_planning import *

#import zmqRemoteApi and make an instance of the RemoteAPIClient class
from coppeliasim_zmqremoteapi_client import RemoteAPIClient

client = RemoteAPIClient()
sim = client.getObject('sim')


# Get necessary objects from CoppeliaSim

In [3]:
frame_handles = [sim.getObject(f'/Frame{i}') for i in range(7)]
joint_handles = [sim.getObject(f'/joint{i+1}') for i in range(6)]
target_dummy = sim.getObject('/Target_Dummy')

joint_types = ['revolute' for i in range(6)]

# Make sure joints are in their zero positions

In [4]:
for i in range(6):
    sim.setJointTargetPosition(joint_handles[i],0)

# Forward Kinematics

## Choose the desired joint angles

In [ ]:
# Case 1
q = np.array([90,-45,90,30,-60,90])*np.pi/180

In [ ]:
# Case 2
q = np.array([-45,90,-30,-45,30,0])*np.pi/180

In [ ]:
# Case 3
q = np.array([45,30,90,-30,-90,30])*np.pi/180

In [ ]:
#get DH-table for the required joint angles
dh_table = get_dh_table(q)

# Get Transformation from end-effector frame to base frame
T6_0 = to_base_frame(dh_table)[-1]

# Move joint angles in CoppeliaSim and compare with the analytical results
sim.startSimulation()
for i in range(6):
    sim.setJointTargetPosition(joint_handles[i],q[i])

print("T6_0\n",T6_0.round(3))

time.sleep(2)
print("EE_pose\n",np.array(sim.getObjectMatrix(frame_handles[6],frame_handles[0])).round(3).reshape(3,4))

# Inverse Kinematics

In [5]:
#initial joint angles to make sure robot is out of singularity
q_current = [ 90*np.pi/180, 0, -90*np.pi/180, 0 , 90*np.pi/180,0*np.pi/180] 

max_iter = 30 #max iterations of IK alogorithm

sim.startSimulation()

# move robot to an initial pose away from the robot's singularity
for i in range(6):
    sim.setJointTargetPosition(joint_handles[i],q_current[i])

time.sleep(2)

## Follow target dummy in real-time

In [ ]:
#move robot end-effector with the target-dummy in real-time
while(1):
    n = 0

    # desired pose is the same as the target-dummy
    desired_pose =np.array(sim.getObjectMatrix(target_dummy,frame_handles[0])).round(3).reshape(3,4)
    
    #get new joint angles and error
    q_current , error = ik(q_current,desired_pose,joint_types)

    # Apply the Newton-Raphson method until the error is less than 0.0001 as long as
    # the number of iterations is less than the maximum allowed
    while(error > 0.001):
        if(n < max_iter):
            n += 1
            for i in range(6):
                sim.setJointTargetPosition(joint_handles[i],q_current[i])
            
            q_current , error = ik(q_current,desired_pose,joint_types)
        else:
            break


### Choose desired pose to go to

In [ ]:
p1 = sim.getObjectHandle('/p1')
desired_pose = np.array(sim.getObjectMatrix(p1,frame_handles[0])).round(3).reshape(3,4)

In [ ]:
p2 = sim.getObjectHandle('/p2')
desired_pose = np.array(sim.getObjectMatrix(p2,frame_handles[0])).round(3).reshape(3,4)

In [ ]:
p3 = sim.getObjectHandle('/p3')
desired_pose = np.array(sim.getObjectMatrix(p3,frame_handles[0])).round(3).reshape(3,4)

In [ ]:
p4 = sim.getObjectHandle('/p4')
desired_pose = np.array(sim.getObjectMatrix(p4,frame_handles[0])).round(3).reshape(3,4)

In [ ]:
p5 = sim.getObjectHandle('/p5')
desired_pose = np.array(sim.getObjectMatrix(p5,frame_handles[0])).round(3).reshape(3,4)

In [6]:
L1 = sim.getObject('/L1')
desired_pose = np.array(sim.getObjectMatrix(L1,frame_handles[0])).round(3).reshape(3,4)

### Apply inverse kinematics to reach desired Pose

In [7]:
#get new joint angles and error
q_current , error = ik(q_current,desired_pose,joint_types)
n = 0

while(error > 0.001):
        if(n < max_iter):
            n += 1     
            #Move joints to new joint angles
            for i in range(6):
                sim.setJointTargetPosition(joint_handles[i],q_current[i])
            
            q_current , error = ik(q_current,desired_pose,joint_types)

        else:
            break
            
# Get Transformation from end-effector frame to base frame
dh_table = get_dh_table(q_current)
Ti_1_0 = to_base_frame(dh_table) 
T6_0 = Ti_1_0[-1]

print("T6_0\n",T6_0.round(2))

time.sleep(2)
print("desired_pose\n",desired_pose.round(2))

T6_0
 [[ 1.    0.    0.   -0.03]
 [ 0.   -0.    1.    0.42]
 [ 0.   -1.   -0.    0.39]
 [ 0.    0.    0.    1.  ]]
desired_pose
 [[ 1.    0.    0.   -0.02]
 [-0.   -0.    1.    0.42]
 [ 0.   -1.   -0.    0.39]]


# Linear Interpolation

In [8]:
L1 = sim.getObject('/L1')
L2 = sim.getObject('/L2')
L3 = sim.getObject('/L3')

In [9]:
seg = 1

while(sim.getSimulationTime()<6*60):
    
    if(seg==1):
        time.sleep(1.5)
        initial_pose = np.array(sim.getObjectMatrix(L1,frame_handles[0])).round(3).reshape(3,4)
        final_pose = np.array(sim.getObjectMatrix(L2,frame_handles[0])).round(3).reshape(3,4)
    elif(seg==2):
        initial_pose = np.array(sim.getObjectMatrix(L2,frame_handles[0])).round(3).reshape(3,4)
        final_pose = np.array(sim.getObjectMatrix(L3,frame_handles[0])).round(3).reshape(3,4)
    elif(seg==3):
        initial_pose = np.array(sim.getObjectMatrix(L3,frame_handles[0])).round(3).reshape(3,4)
        final_pose = np.array(sim.getObjectMatrix(L1,frame_handles[0])).round(3).reshape(3,4)
    
    if(seg<3):
        seg = seg+1
    else:
        seg = 1
    
    poses = generate_path(initial_pose,final_pose,0.02,2)

    #Apply Inverse Kinematics
    for i in range(len(poses)):

        desired_pose = poses[i]
        #get new joint angles and error
        q_current , error = ik(q_current,desired_pose,joint_types)
        n = 0
        max_iter = 3
        while(error > 0.001):
                if(n < max_iter):
                    n += 1
                    #Move joints to new joint angles
                    for i in range(6):
                        sim.setJointTargetPosition(joint_handles[i],q_current[i])

                    q_current , error = ik(q_current,desired_pose,joint_types)
                else:
                    break
        
sim.stopSimulation()